In [1]:
!pip install graphviz

In [1]:
from utils import *

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
grid = np.array([[2, 0, 0, 2, 0],
                [2, 0, 2, 0, 2],
                [0, 2, 0, 0, 0],
                [0, 1, 1, 0, 0],
                [0, 1, 1, 1, 0],
                [1, 1, 1, 1, 1]])
player_to_move = 1

board = Board(5, grid)
state = State(board=board, player_to_move = player_to_move)
tree = Tree(model)   
p_target = tree.think(state, 1000, 0.1, show=True)

6  B . . B .
5  B . B . B
4  . B . . .
3  . W W . .
2  . W W W .
1  W W W W W

   a b c d e

1.00 sec. best 47. q = 0.9667. n = 492 / 887. pv = 47


In [6]:
show_mcts_exploration(tree, state, min_visit_count = 10)

100%|██████████| 418/418 [00:03<00:00, 134.03it/s]


In [3]:
# Save images
from PIL import Image
from tqdm import tqdm

names = []


for path in tqdm(tree.nodes.keys()):
    state_ = copy.deepcopy(state)
    if len(path) > 0:
        actions = path.split('|')[1].split(' ') 
        for action in actions:
            state_.play(int(action))       
        parent = 'root' if len(actions) == 1 else '-'.join(actions[:-1])
        name = 'namesplit_' + '-'.join(actions)+'namesplit_' + parent + 'namesplit_' + str(tree.nodes[path].n_all)+ '.png'
    else:
        name = 'root.png'
    names.append(name)
    arr = get_image_array(state_)
    
    img = Image.fromarray((arr).astype(np.uint8))
    img = img.resize((img.size[0] // 6, img.size[0] // 6))
    img.save('figs/'+name)
    


100%|██████████| 246/246 [00:01<00:00, 130.43it/s]


In [5]:
from graphviz import Graph
g = Graph('G', 
          node_attr={'color': 'lightblue2',
                    'shape':'plaintext'},
          filename='process.gv', 
          engine='dot', 
          format='svg')
g.attr(fontsize='100')
min_visit_count = 5
# Create nodes
for name in names:
    if name != 'root.png':
        _, node_name, parent_name, visit = name.replace('.png','').split('namesplit_')
        if int(visit) < min_visit_count:
            continue
    else:
        node_name = 'root'
    g.node(node_name,label="",image='figs/'+name)
    
# Create edges
for name in names:
    if name != 'root.png':
        _, node_name, parent_name, visit = name.replace('.png','').split('namesplit_')
        if int(visit) < min_visit_count:
            continue
        g.edge(parent_name, node_name, label = visit)
    
g.view()

'process.gv.svg'